In [1]:
from os import environ

environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ["HTTP_PROXY"] = "http://localhost:7890"
environ["HTTPS_PROXY"] = "http://localhost:7890"
environ["OPENAI_API_BASE"] = "http://10.1.3.28:8990/v1"
environ["OPENAI_API_KEY"] = "sk-"

from retrieval_qa_benchmark.models import *
from retrieval_qa_benchmark.datasets import *
from retrieval_qa_benchmark.transforms import *
from retrieval_qa_benchmark.protocols import *
from retrieval_qa_benchmark.utils.registry import REGISTRY

print(str(REGISTRY))

Datasets:
	hotpot_qa
	mmlu
	wikihop
Transforms:
	base
	mcsa_prompt
	mcsa_fewshot_mmlu
	mcma_prompt
Models:
	gpt35
	chatgpt35
Evaluators:
	mcma
	mcsa


/Users/mpsk/Library/Caches/pypoetry/virtualenvs/retrieval-qa-benchmark-OUwPJWBa-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/mpsk/Library/Caches/pypoetry/virtualenvs/retrieval-qa-benchmark-OUwPJWBa-py3.11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
import yaml

config = yaml.safe_load(open("../config/mmlu-llama2-remote.yaml"))
evaluator = REGISTRY.Evaluators["mcsa"].from_config(config)
print(evaluator.transform)

Converting dataset...: 100%|██████████| 324/324 [00:00<00:00, 37730.92it/s]

chain=[MultipleChoiceTransform(sep_chr='\n', prompt_prefix='Please answer with the letter of the correct answer.\n', prompt_suffix='Answer: ')]


In [4]:
evaluator.transform(evaluator.dataset[0])

QARecord(id='4e8db4a928333356588d2b89477e5d5bcb95532cd8afa355456ac94e13fd1849', question='The following are multiple choice questions (with answers).\n\nQuestion: Unlike most other early civilizations, Minoan culture shows little evidence of:\nChoices: A. trade.\tB. warfare.\tC. the development of a common religion.\tD. conspicuous consumption by elites.\nAnswer: ', answer='conspicuous consumption by elites.', type='MCSA', choices=['trade.', 'warfare.', 'the development of a common religion.', 'conspicuous consumption by elites.'])

In [5]:
matched = evaluator()

Evaluating: 100%|██████████| 324/324 [12:57<00:00,  2.40s/it]
2023-08-07 11:05:47.727 | INFO     | retrieval_qa_benchmark.protocols.base:__call__:60 - Evaluation finished! Executed Evaluator:<class 'retrieval_qa_benchmark.protocols.mcsa.MCSAEvaluator'> on Dataset:cais/mmlu.prehistory-test with Model:llama2. Accuracy: 55.86%


In [6]:
with open('results.jsonl', 'w') as f:
    f.write('\n'.join([r.model_dump_json() for r in matched[1]]))

In [ ]:
for dname, dset in REGISTRY.Datasets.items():
    print(f"Loading {dname}...")
    dset.build()

In [ ]:
from retrieval_qa_benchmark.schema import QAPrediction
mismatched = [pred for pred in matched if not pred.matched]

In [ ]:
print(mismatched[0].question)
print(mismatched[0].answer)
print(mismatched[0].pred)